In [3]:
import tensorflow as tf
try: [tf.config.experimental.set_memory_growth(gpu, True) for gpu in tf.config.experimental.list_physical_devices("GPU")]
except: pass

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from mltu.tensorflow.dataProvider import DataProvider
from mltu.tensorflow.losses import CTCloss
from mltu.tensorflow.callbacks import Model2onnx, TrainLogger
from mltu.tensorflow.metrics import CWERMetric

from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate
from mltu.annotations.images import CVImage

from model import train_model
from configs import ModelConfigs

import os
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to="Datasets"):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)


if not os.path.exists(os.path.join("Datasets", "captcha_images_v2")):
    download_and_unzip("https://github.com/AakashKumarNain/CaptchaCracker/raw/master/captcha_images_v2.zip",
                       extract_to="Datasets")

2024-02-28 11:45:15.829934: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-28 11:45:15.830006: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-28 11:45:15.830043: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mahedi-HP-Laptop-15s-fq5xxx): /proc/driver/nvidia/version does not exist


librosa not found. Please install it with `pip install librosa` if you plan to use it.
librosa not found. Please install it with `pip install librosa` if you plan to use it.
librosa not found. Please install it with `pip install librosa` if you plan to use it.


In [4]:
dataset, vocab, max_len = [], set(), 0
captcha_path = os.path.join("Datasets", "captcha_images_v2")
for file in os.listdir(captcha_path):
    file_path = os.path.join(captcha_path, file)
    label = os.path.splitext(file)[0] # Get the file name without the extension
    dataset.append([file_path, label])
    vocab.update(list(label))
    max_len = max(max_len, len(label))

In [9]:
import pandas as pd

pd.DataFrame(dataset)

,0,1
0,Datasets/captcha_images_v2/6ydyp.png,6ydyp
1,Datasets/captcha_images_v2/wf684.png,wf684
2,Datasets/captcha_images_v2/befbd.png,befbd
3,Datasets/captcha_images_v2/b4ncn.png,b4ncn
4,Datasets/captcha_images_v2/cnmnn.png,cnmnn
...,...,...
1035,Datasets/captcha_images_v2/wc2bd.png,wc2bd
1036,Datasets/captcha_images_v2/n8ydd.png,n8ydd
1037,Datasets/captcha_images_v2/ncww7.png,ncww7
1038,Datasets/captcha_images_v2/yd3m3.png,yd3m3


In [8]:
len(vocab)

19

In [7]:
max_len

5